# Anwendung 2: Topic Modeling

Eine weitere Anwendung von Vector-Space-Modellen ist das Topic Modeling. Es zielt auf die Identifikation von »topics« in einem Corpus. Die Berechnung dieser topics basiert dabei auf ihrer Verteilung in den Dokumenten des Corpus und damit letztlich auf Wort-Kookkurrenzen. Ob die ermittelten topics tatsächlich *Themen* im inhaltlichen Sinne, oder eher »Diskursstränge«, Wort-Cluster o.ä. sind, bleibt der inhaltlichen Interpretation überlassen.

Weiter kann an dieser Stelle nicht auf die Hintergründe des Topic Modeling eingegangen werden. Für die weitere Lektüre sei daher dieser Beitrag empfohlen:

Brett, Megan R. (2012): „Topic Modeling: A Basic Introduction“, Journal of Digital Humanities 2/1, http://journalofdigitalhumanities.org/2-1/topic-modeling-a-basic-introduction-by-megan-r-brett/.

Für das Topic Modeling muss der Text nur minimal aufbereitet werden. Im Gegensatz zu der `TextCorpus`-Klasse, die für die Keyword Extraction verwendet wurde, wurden aber zwei Details verändert, die die Qualität der erzeugten Topics verbessern:

* Für die Tokenisierung, also die Identifikation von Worten, wird nicht auf `TextBlob` zurückgegriffen, sondern der reguläre Ausdruck `\w+` verwendet. Durch die Begrenzung auf alphanumerische Zeichen werden etwa Satzzeichen wie Bindestriche oder Anführungszeichen automatisch aussortiert.
* Es wird ein Stoppwortfilter verwendet, der auf einer vorgegebenen Liste basiert.

Diese beiden Punkte sind in den Methoden `tokenize()` und `filter()` umgesetzt. Die Methode `get_texts()` wurde entsprechend angepasst. Zusätzlich sind noch zwei Details umgesetzt, die die Verarbeitungsgeschwindigkeit erhöhen: So wird die CSV-Tabelle nur einmal eingelesen, anstatt bei jedem Durchlauf neu geladen zu werden. Ebenso wird die Anzahl der Dokumente im Corpus, die für manche Berechnungen notwendig ist, zwischengespeichert. Diese beiden Punkte sind aber nicht zwingend erforderlich.

In [1]:
import re

from gensim.corpora.textcorpus import TextCorpus
from textblob_de import TextBlobDE as TextBlob
from textblob_de import PatternParser
import pandas as pd

class CSVCorpus(TextCorpus):
    """Read corpus from a csv file."""

    def tokenize(self, text):
        words = re.findall('\w+', text.lower(), re.U)
        return words

    def filter(self, tokens, stopwords):
        return [token for token in tokens if not token in stopwords]

    def get_texts(self):
        with open('../Daten/stopwords.txt') as stopwordfile:
            stopwords = stopwordfile.read().splitlines()
        table = self.gettable()
        for text in table['text']:
            tokens = self.tokenize(text)
            yield self.filter(tokens, stopwords)

    def gettable(self):
        if not hasattr(self, 'table'):
            with self.getstream() as csvfile:
                self.table = pd.read_csv(csvfile, parse_dates=['date'], encoding='utf-8')
        return self.table

    def __len__(self):
        if not hasattr(self, 'length'):
            # Cache length
            self.length = len(self.gettable())
        return self.length

Alternativ kann auch auf die Verfahren der Lemmatisierung und Wortartenfilterung zurückgegriffen werden, die in den vorherigen Einheiten besprochen wurden. Dazu werden die beiden Methoden `tokenzie()` und `filter()` überschrieben. Damit dauert die Verarbeitung aber deutlich länger, und es ist unklar, ob die Qualität der Analyse dadurch zwingend steigt. Für ein vergleichendes Experiment kann diese Version aber verwendet werden.

In [2]:
from string import punctuation
from collections import namedtuple

class LemmatizedCSVCorpus(CSVCorpus):
    """Read corpus from a csv file."""

    def tokenize(self, text):
        text = text.replace('\xa0', ' ')  # Ersetze "non-breaking space" durch normales Leerzeichen
        text = re.sub('[„“”‚‘’–]', '', text, re.U)  # Entferne Anführungzeichen und Gedankenstriche
        blob = TextBlob(text, parser=PatternParser(lemmata=True))
        parse = blob.parse()
        fieldnames = [tag.replace('-', '_') for tag in parse.tags]
        Token = namedtuple('Token', fieldnames)
        tokens = [Token(*token.split('/', 4)) for token in parse.split(' ')]
        return tokens

    def filter(self, tokens, stopwords):
        result = []
        for token in tokens:
            pos = token.part_of_speech[0:2]
            word = token.word
            if not word.lower() in stopwords and not word in punctuation:
                if pos == 'NN':
                    result.append(token.lemma.title())
                else:
                    result.append(token.lemma)
        return result

In [3]:
corpus = CSVCorpus('../Daten/Reden.csv')
len(corpus.dictionary)

56551

In der nicht lemmatisierten Form enthält das Corpus nun also 56551 unterschiedliche Wörter. Besonders häufige ebenso wie besonders seltene Wörter können dabei die Analyse negativ beeinflussen. Gensim stellt eine Methode bereit, mit der das Corpus um diese Extremwerte bereinigt werden kann. (Erst im zweiten Schritt `compactify()` werden dann die gelöschten Extremwerte tatsächlich aus dem Corpus entfernt.)

In [4]:
corpus.dictionary.filter_extremes()
corpus.dictionary.compactify()
len(corpus.dictionary)

12470

Nach dem Filtern bleiben noch 12470 Einträge übrig.

Gensim enthält eine Implementierung des Topic-Modeling-Verfahrens »LDA« (neben anderen). Diese Version ist dabei für sehr große Corpora optimiert, die resultierenden Topics sind aber leider oft nicht sehr leicht zu interpretieren. Daher soll hier die Implementierung aus dem Python-Paket »lda« verwendet werden. Sie arbeitet nicht direkt mit dem TextCorpus-Format von gensim, kann aber eine *sparse matrix* einlesen, die mit gensim erzeugt wird.

In [5]:
from gensim.matutils import corpus2csc

corpus_matrix = corpus2csc(corpus)
corpus_matrix.shape

(12470, 793)

In [6]:
corpus_matrix = corpus_matrix.transpose()
corpus_matrix.shape

(793, 12470)

Aus technischen Gründen muss die Matrix nun noch in ein bestimmtes Zahlenformat (Kommazahl zu Ganzzahl) konvertiert werden.

In [7]:
corpus_matrix = corpus_matrix.astype(int)

Ähnlich wie bei der Keyword Extraction wird hier im ersten Schritt ein Modell erzeugt, das vor allem bestimmte Parameter für die Berechnung speichert. Der wichtigste ist hierbei die Anzahl der Topics. Diese muss vorgegeben werden und kann nicht vom Algorithmus selbst bestimmt werden. 20 ist oft ein guter Ausgangswert, man sollte aber mit verschiedenen Werten experimentieren und die Ergebnisse vergleichen.

In [8]:
import lda
lda_model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
lda_model.fit(corpus_matrix)

*Hinweis:* Um die Ergebnisse etwas übersichtlicher darzustellen, ist eine gewisse Formatierung der Ausgabe nützlich. Dies könnte etwa mit HTML realisiert werden. Etwas einfacher ist das minimalistische Textformat »Markdown«, das etwa Zeilen mit vorangestellten Sternchen `*` in Aufzählungslisten umwandelt. Um dies einfacher zu nutzen, wird hier eine kleine Hilfsfunktion definiert. Zu Details siehe die [Syntax-Beschreibung](http://www.daringfireball.net/projects/markdown/syntax).

In [9]:
from IPython.nbconvert.filters.markdown import markdown2html

class MD(str):
    def _repr_html_(self):
        return markdown2html(self)

MD('Das ist ein **Test!**')

'Das ist ein **Test!**'

Das Ergebnisformat von LDA sind Matrizen im numpy-Format. Diese sind sehr effizient und bieten eine Reihe von Berechnungsmöglichkeiten, sie sind jedoch auf den ersten Blick nicht ganz leicht zu verstehen. Der folgende Code wurde aus der Dokumentation des lda-Pakets übernomme und ein wenig angepasst. Für den Moment soll die Beschreibung ausreichen, dass hierüber für jedes Topic die einflussreichsten 20 Wörter ausgegeben werden.

In [10]:
import numpy as np
vocab = [corpus.dictionary[i] for i in range(len(corpus.dictionary))]
topic_word = lda_model.topic_word_  # model.components_ also works
n_top_words = 20
topics = [np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
          for topic_dist in topic_word]
result = ''
for i, topic_words in enumerate(topics):
    result += '* **Topic {}:** {}\n'.format(i, ' '.join(topic_words))
MD(result)

'* **Topic 0:** forschung wissenschaft professor universität bildung wissenschaftler wissenschaftliche chemie akademie wissenschaftlichen gesellschaft max wissenschaften universitäten forschungseinrichtungen gemeinschaft 000 erkenntnisse forscher\n* **Topic 1:** deutlich eu entwicklung ziel rolle bleibt vergangenen rahmen insbesondere zusammenarbeit kommission insgesamt wirtschaftlichen wichtiger maßnahmen schritt erfolg wichtige ausdrücklich\n* **Topic 2:** film filme berlinale kinos the wort kino gesprochene films abend a künstler of ffa anrede and herzlich filmförderung to\n* **Topic 3:** sicherheit bundeswehr afghanistan freiheit ukraine soldaten nato herausforderungen frieden verantwortung staaten einsatz israel syrien terrorismus lösung internationale iran internationalen\n* **Topic 4:** union europäische mitgliedstaaten europas krise gemeinsamen gemeinsame wirtschafts währungsunion staaten kontinent währung sprechen europäer wohlstand frankreich entwicklung wettbewerbsfähigkeit freiheit\n* **Topic 5:** freiheit kultur gesellschaft kunst demokratie politischen kirche politische politik buch kirchen sinne religion künstler werte blick lässt bilder lebens\n* **Topic 6:** kunst museen museum ausstellung berlin künstler geschichte kulturellen gesprochene humboldt museums werke kulturnation zeiten wort sammlung goethe anrede künste\n* **Topic 7:** sozusagen fragen dingen überhaupt richtig gesprochen stellen lange thema all diskussion bekommen worden stück 20 führen finde zeigt genau\n* **Topic 8:** geschichte freiheit ddr einheit erinnern mauer 20 1989 erinnerung demokratie tag ehemaligen leid bundesrepublik helmut teilung vergessen versöhnung flucht\n* **Topic 9:** internet medien digitalen vielfalt welle öffentlich digitale netz urheber zeitungen urheberrecht zeitalter kreativen informationen qualität rechtlichen journalisten digitalisierung schutz\n* **Topic 10:** unternehmen wirtschaft bereich entwicklung industrie rahmenbedingungen it chancen messe thema branche sprechen wettbewerb elektromobilität ausbildung insgesamt forschung ministerpräsident infrastruktur\n* **Topic 11:** stiftung berlin ausstellung haus wort aufarbeitung bund anrede berliner ns nationalsozialistischen gedenkstätte opfer historischen bedeutung 000 einrichtungen engagement gesprochene\n* **Topic 12:** prozent krise wachstum wettbewerbsfähigkeit banken unternehmen millionen wirtschaft richtig marktwirtschaft internationalen milliarden vertrauen wirtschaftskrise schulden situation sozialen forschung weltweit\n* **Topic 13:** gesellschaft kinder familie frauen familien beruf integration junge jungen thema eltern lernen alter schule liebe wandel bildung kommunen älteren\n* **Topic 14:** politik verantwortung weise recht sprechen bedeutet verstehen angesichts raum aufgabe gerne weit fragen eigene bürgerinnen wort großer aufgaben kraft\n* **Topic 15:** prozent energien erneuerbaren bereich thema milliarden nächsten investitionen augenblick punkt 40 bedeutet anbelangt bedeutung energiewende bund sprechen erreichen investieren\n* **Topic 16:** kultur bund kulturelle millionen kulturellen bundes haus bildung förderung ort gesellschaft projekte bedeutung sachsen kommunen engagement 000 vielfalt 1\n* **Topic 17:** liebe deutsch berlin herzlich namen glück zeigt selbstverständlich blick preis präsident freude genau gemeinsame abend junge gemeinsamen 50 vertrauen\n* **Topic 18:** nachhaltigkeit entwicklung internationalen g7 ziel erreichen ziele konferenz landwirtschaft nachhaltige klimaschutz weltweit nationen un vereinten umwelt internationale globale ressourcen\n* **Topic 19:** china zusammenarbeit staaten unternehmen beziehungen amerika vereinigten wirtschaft deutsch partnerschaft freihandelsabkommen kooperation russland region abkommen partner gesprochen freue wirtschaftliche\n'

Diese Topics sind zunächst einmal probabilistisch identifizierte Wort-Cluster. Bei genauerer Betrachtung lässt sich aber für die meisten Topics ein Eindruck gewinnen, welches Thema die ausgegebenen Worte umreißen. Topic 0 etwa beschreibt die Themenfelder Universität und Forschung, Topic 2 dagegen den Bereich Film und Kino. Topic 1 dagegen lässt sich auf den ersten Blick weniger leicht interpretieren.

Neben den Topics selbst gibt LDA auch eine Zuordnung von Topics zu Dokumenten aus. Es lassen sich also auch für jedes Dokument die relevantesten Topics ausgeben, die das Dokument beschreiben. Die ist ähnlich wie bei der Keyword Extraction, nur dass ganze Topics und nicht einzelne Schlüsselwörter zur Beschreibung herangezogen werden.

In [11]:
data = pd.read_csv("../Daten/Reden.csv", parse_dates=['date'], encoding='utf-8')
titles = data['title']

doc_topic = lda_model.doc_topic_
result = ''
for i in range(10):
    result += '\n\n**{}**\n\n'.format(titles[i])
    for topic in doc_topic[i].argsort()[:-4:-1]:
        result += ' * _Topic {}:_ {}\n'.format(topic, ' '.join(topics[topic]))
MD(result)

'\n\n**Rede von Bundeskanzlerin Merkel beim Empfang der Preisträgerinnen und Preisträger des 50. Bundeswettbewerbs „Jugend forscht“ am 30. September 2015**\n\n * _Topic 0:_ forschung wissenschaft professor universität bildung wissenschaftler wissenschaftliche chemie akademie wissenschaftlichen gesellschaft max wissenschaften universitäten forschungseinrichtungen gemeinschaft 000 erkenntnisse forscher\n * _Topic 7:_ sozusagen fragen dingen überhaupt richtig gesprochen stellen lange thema all diskussion bekommen worden stück 20 führen finde zeigt genau\n * _Topic 14:_ politik verantwortung weise recht sprechen bedeutet verstehen angesichts raum aufgabe gerne weit fragen eigene bürgerinnen wort großer aufgaben kraft\n\n\n**Rede von Bundeskanzlerin Merkel beim Global Leaders‘ Meeting on Achieving Gender Equality and Women’s Empowerment: „A Commitment to Action“ am 27. September 2015**\n\n * _Topic 13:_ gesellschaft kinder familie frauen familien beruf integration junge jungen thema eltern lernen alter schule liebe wandel bildung kommunen älteren\n * _Topic 1:_ deutlich eu entwicklung ziel rolle bleibt vergangenen rahmen insbesondere zusammenarbeit kommission insgesamt wirtschaftlichen wichtiger maßnahmen schritt erfolg wichtige ausdrücklich\n * _Topic 18:_ nachhaltigkeit entwicklung internationalen g7 ziel erreichen ziele konferenz landwirtschaft nachhaltige klimaschutz weltweit nationen un vereinten umwelt internationale globale ressourcen\n\n\n**Rede von Bundeskanzlerin Merkel beim gemeinsamen Side Event der Bundesrepublik Deutschland, des Königreichs Norwegen und der Republik Ghana „Securing a healthy future“ am 26. September 2015**\n\n * _Topic 7:_ sozusagen fragen dingen überhaupt richtig gesprochen stellen lange thema all diskussion bekommen worden stück 20 führen finde zeigt genau\n * _Topic 1:_ deutlich eu entwicklung ziel rolle bleibt vergangenen rahmen insbesondere zusammenarbeit kommission insgesamt wirtschaftlichen wichtiger maßnahmen schritt erfolg wichtige ausdrücklich\n * _Topic 18:_ nachhaltigkeit entwicklung internationalen g7 ziel erreichen ziele konferenz landwirtschaft nachhaltige klimaschutz weltweit nationen un vereinten umwelt internationale globale ressourcen\n\n\n**Rede von Bundeskanzlerin Merkel beim Arbeitsmittagessen des UN Private Sector Forum 2015 am 26. September 2015**\n\n * _Topic 18:_ nachhaltigkeit entwicklung internationalen g7 ziel erreichen ziele konferenz landwirtschaft nachhaltige klimaschutz weltweit nationen un vereinten umwelt internationale globale ressourcen\n * _Topic 7:_ sozusagen fragen dingen überhaupt richtig gesprochen stellen lange thema all diskussion bekommen worden stück 20 führen finde zeigt genau\n * _Topic 19:_ china zusammenarbeit staaten unternehmen beziehungen amerika vereinigten wirtschaft deutsch partnerschaft freihandelsabkommen kooperation russland region abkommen partner gesprochen freue wirtschaftliche\n\n\n**Rede von Bundeskanzlerin Merkel beim High Level Event der United Nations Development Group (UNDG) „From Global Conversation to Global Action – Making the Sustainable Development Goals Work“ am 25. September 2015**\n\n * _Topic 18:_ nachhaltigkeit entwicklung internationalen g7 ziel erreichen ziele konferenz landwirtschaft nachhaltige klimaschutz weltweit nationen un vereinten umwelt internationale globale ressourcen\n * _Topic 7:_ sozusagen fragen dingen überhaupt richtig gesprochen stellen lange thema all diskussion bekommen worden stück 20 führen finde zeigt genau\n * _Topic 1:_ deutlich eu entwicklung ziel rolle bleibt vergangenen rahmen insbesondere zusammenarbeit kommission insgesamt wirtschaftlichen wichtiger maßnahmen schritt erfolg wichtige ausdrücklich\n\n\n**Rede von Bundeskanzlerin Merkel bei der Eröffnung des UN-Gipfels zur Verabschiedung der Agenda 2030 für nachhaltige Entwicklung am 25. September 2015 **\n\n * _Topic 18:_ nachhaltigkeit entwicklung internationalen g7 ziel erreichen ziele konferenz landwirtschaft nachhaltige klimaschutz weltweit n

Dabei ist zunächst auffällig, dass die Topics 1 und/oder 7 relativ häufig auftauchen. Gemeinsam mit ihrer inhaltlichen Vagheit ergibt sich der Eindruck, dass es sich weitgehend um Residual-Topics handelt, die relativ unspezifische, aber regelmäßig auftretenden Wörter gruppieren. Die anderen Topics geben dagegen einen relativ guten Einblick in die inhaltlichen Schwerpunkte der Texte.

Um später erneut auf die Ergebnisse zurückgreifen zu können, wird das berechnete Modell gespeichert.

In [12]:
from pickle import dump

with open('../Daten/topicmodel20.pickle', 'wb') as picklefile:
    dump(lda_model, picklefile)